In [1]:
# Import libraries
import requests
import json
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

In [131]:
def get_season_list(start_year=2017):
    url = 'http://ergast.com/api/f1/seasons.json?limit=1000'
    response = requests.get(url)
    data = response.json()
    seasons = data['MRData']['SeasonTable']['Seasons']
    
    df_seasons = pd.DataFrame(seasons)
    
    df_seasons = df_seasons[df_seasons['season'].astype(int) >= start_year]
    
    return df_seasons.reset_index(drop=True)

In [132]:
def get_race_schedule(start_year=2017, end_year=2024):
    all_races = []
    for year in range(start_year, end_year + 1):
        url = f'http://ergast.com/api/f1/{year}.json'
        response = requests.get(url)
        races = response.json()['MRData']['RaceTable']['Races']
        for race in races:
            race_info = {
                'season': race['season'],
                'round': race['round'],
                'raceName': race['raceName'],
                'date': race['date'],
                'time': race['time'],
                'circuitId': race['Circuit']['circuitId'],
                'circuitName': race['Circuit']['circuitName'],
                'location-locality': race['Circuit']['Location']['locality'],
                'location-country': race['Circuit']['Location']['country'],
                'location-lat': race['Circuit']['Location']['lat'],
                'location-long': race['Circuit']['Location']['long'],
            }
            all_races.append(race_info)
    
    df_race_schedule = pd.DataFrame(all_races)
    
    df_race_schedule['date'] = pd.to_datetime(df_race_schedule['date'])
    
    return df_race_schedule

In [4]:
def get_rounds_in_season(year):
    url = f'http://ergast.com/api/f1/{year}.json'
    response = requests.get(url)
    return len(response.json()['MRData']['RaceTable']['Races'])

def fetch_race_results(year, round_num):
    url = f'http://ergast.com/api/f1/{year}/{round_num}/results.json?limit=1000'
    response = requests.get(url)
    data = response.json()['MRData']['RaceTable']['Races']
    
    if not data:
        return []
    
    if data:
        race = data[0]
        race_results = []
        for result in race['Results']:
            result_info = {
                'season': race['season'],
                'round': race['round'],
                'raceName': race['raceName'],
                'date': race['date'],  # Useful for analysis later
                'driverId': result['Driver']['driverId'],
                'constructorId': result['Constructor']['constructorId'],
                'grid': result['grid'],
                'position': result.get('position', None),  # Sometimes position can be missing
                'status': result['status'],
                'points': result['points'],
                'time-ms': result['Time']['millis'] if 'Time' in result else None,
                'time-hrmins': result['Time']['time'] if 'Time' in result else None,
                'Max Avg Speed Unit': result['FastestLap']['AverageSpeed']['units'] if 'FastestLap' in result else None,
                'Max Avg Speed': result['FastestLap']['AverageSpeed']['speed'] if 'FastestLap' in result else None
            }
            race_results.append(result_info)
    
    return race_results

def get_race_results(start_year, end_year):
    all_results = []

    with ThreadPoolExecutor(max_workers=12) as executor:
        futures = []
        
        for year in range(start_year, end_year + 1):
            num_rounds = get_rounds_in_season(year)
            
            for round_num in range(1, num_rounds + 1):
                futures.append(executor.submit(fetch_race_results, year, round_num))
        
        for future in futures:
            result = future.result()
            if result:
                all_results.extend(result)

    df_race_results = pd.DataFrame(all_results)
    
    df_race_results['date'] = pd.to_datetime(df_race_results['date'])
    
    return df_race_results

In [139]:
def fetch_qualifying_results(year, round_num):
    url = f'http://ergast.com/api/f1/{year}/{round_num}/qualifying.json?limit=1000'
    response = requests.get(url)
    data = response.json()['MRData']['RaceTable']['Races']
    
    if data:
        race = data[0]
        quali_results = []
    
        for quali in race['QualifyingResults']:
            quali_info = {
                'season': race['season'],
                'round': race['round'],
                'raceName': race['raceName'],
                'date': race['date'],  # Include the race date for better analysis
                'driverId': quali['Driver']['driverId'],
                'constructorId': quali['Constructor']['constructorId'],
                'position': quali.get('position', None),
                'Q1': quali.get('Q1', None),
                'Q2': quali.get('Q2', None),
                'Q3': quali.get('Q3', None)
            }
            quali_results.append(quali_info)
    
        return quali_results

def get_qualifying_results(start_year, end_year):
    all_results = []

    with ThreadPoolExecutor(max_workers=24) as executor:
        futures = []
        
        for year in range(start_year, end_year + 1):
            num_rounds = get_rounds_in_season(year)
            
            for round_num in range(1, num_rounds + 1):
                futures.append(executor.submit(fetch_qualifying_results, year, round_num))
        
        for future in futures:
            result = future.result()
            if result:
                all_results.extend(result)

    df_quali_results = pd.DataFrame(all_results)
    
    df_quali_results['date'] = pd.to_datetime(df_quali_results['date'])
    
    return df_quali_results

df_quali_results = get_qualifying_results(2017, 2024)

In [144]:
def fetch_sprint_results(year, round_num):
    url = f'http://ergast.com/api/f1/{year}/{round_num}/sprint.json?limit=1000'
    response = requests.get(url)
    data = response.json()['MRData']['RaceTable']['Races']
    
    if data:
        race = data[0]
        sprint_results = []
        for sprint in race['SprintResults']:
            quali_info = {
                'season': race['season'],
                'round': race['round'],
                'raceName': race['raceName'],
                'date': race['date'],
                'driverId': sprint['Driver']['driverId'],
                'constructorId': sprint['Constructor']['constructorId'],
                'grid': sprint['grid'],
                'laps': sprint['laps'],
                'position': sprint.get('position', None),
                'status': sprint['status'],
                'points': sprint['points'],
                'time-ms': sprint['Time']['millis'] if 'Time' in sprint else None,
                'time-minsecmil': sprint['Time']['time'] if 'Time' in sprint else None,
                'fastest-time': sprint['FastestLap']['Time']['time'] if 'Time' in sprint else None,
                }
            sprint_results.append(quali_info)
        return sprint_results

def get_sprint_results(start_year, end_year):
    all_results = []

    with ThreadPoolExecutor(max_workers=24) as executor:
        futures = []
        
        for year in range(start_year, end_year + 1):
            num_rounds = get_rounds_in_season(year)
            
            for round_num in range(1, num_rounds + 1):
                futures.append(executor.submit(fetch_sprint_results, year, round_num))
        
        for future in futures:
            result = future.result()
            if result:
                all_results.extend(result)

    df_sprint_results = pd.DataFrame(all_results)
    
    df_sprint_results['date'] = pd.to_datetime(df_sprint_results['date'])
    
    return df_sprint_results

df_sprint_results = get_sprint_results(2021, 2024)

In [147]:
def get_driver_standings(start_year=2017, end_year=2024):
    all_standings = []
    for year in range(start_year, end_year + 1):
        url = f'http://ergast.com/api/f1/{year}/driverStandings.json?limit=1000'
        response = requests.get(url)
        data = response.json()['MRData']['StandingsTable']['StandingsLists']
        if data:
            standings = data[0]['DriverStandings']
            for standing in standings:
                standing_info = {
                    'season': year,
                    'position': standing['position'],
                    'points': standing['points'],
                    'wins': standing['wins'],
                    'driverId': standing['Driver']['driverId'],
                    'driverName': f"{standing['Driver']['givenName']} {standing['Driver']['familyName']}",
                    'constructorId': standing['Constructors'][0]['constructorId'],
                    'constructorName': standing['Constructors'][0]['name']
                }
                all_standings.append(standing_info)
    df_driver_standings = pd.DataFrame(all_standings)
    return df_driver_standings

df_driver_standings = get_driver_standings(2017, 2024)

In [149]:
def get_constructor_standings(start_year=2017, end_year=2024):
    all_standings = []
    for year in range(start_year, end_year + 1):
        url = f'http://ergast.com/api/f1/{year}/constructorStandings.json?limit=1000'
        response = requests.get(url)
        data = response.json()['MRData']['StandingsTable']['StandingsLists']
        if data:
            standings = data[0]['ConstructorStandings']
            for standing in standings:
                standing_info = {
                    'season': year,
                    'position': standing['position'],
                    'points': standing['points'],
                    'wins': standing['wins'],
                    'constructorId': standing['Constructor']['constructorId'],
                    'constructorName': standing['Constructor']['name'],
                    'nationality': standing['Constructor']['nationality']
                }
                all_standings.append(standing_info)
    df_constructor_standings = pd.DataFrame(all_standings)
    return df_constructor_standings

df_constructor_standings = get_constructor_standings(2017, 2024)

In [150]:
def get_constructor_info():
    url = 'http://ergast.com/api/f1/constructors.json?limit=1000'
    response = requests.get(url)
    constructors = response.json()['MRData']['ConstructorTable']['Constructors']
    df_constructors = pd.DataFrame(constructors)
    return df_constructors

In [1]:
def get_driver_info():
    drivers = []
    limit = 100
    offset = 0

    while True:
        url = f'http://ergast.com/api/f1/drivers.json?limit={limit}&offset={offset}'
        response = requests.get(url)
        data = response.json()['MRData']['DriverTable']['Drivers']
        
        if not data:
            break
        
        drivers.extend(data)
        offset += limit

    df_drivers = pd.DataFrame(drivers)
    
    df_drivers['dateOfBirth'] = pd.to_datetime(df_drivers['dateOfBirth'])
    
    return df_drivers

In [153]:
def get_circuit_info():
    url = 'http://ergast.com/api/f1/circuits.json?limit=1000'
    response = requests.get(url)
    circuits = response.json()['MRData']['CircuitTable']['Circuits']
    
    all_circuits = []
    
    for circuit in circuits:
        circuit_info = {
            'circuitId': circuit['circuitId'],
            'circuitName': circuit['circuitName'],
        }
        
        location = circuit['Location']
        circuit_info.update({
            'lat': location.get('lat', None),
            'long': location.get('long', None),
            'locality': location.get('locality', None),
            'country': location.get('country', None)
        })
        
        all_circuits.append(circuit_info)
    
    df_circuits = pd.DataFrame(all_circuits)
    
    return df_circuits

In [155]:
def get_finishing_status():
    url = 'http://ergast.com/api/f1/status.json?limit=1000'
    response = requests.get(url)
    statuses = response.json()['MRData']['StatusTable']['Status']
    df_status = pd.DataFrame(statuses)
    return df_status

In [157]:
def fetch_pit_results(year, round_num):
    url = f'http://ergast.com/api/f1/{year}/{round_num}/pitstops.json?limit=1000'
    response = requests.get(url)
    data = response.json()['MRData']['RaceTable']['Races']
    if data:
        pitstops = data[0]['PitStops']
        pit_results = []
        for pit in pitstops:
            pit_info = {
                'season': year,
            'round': round_num,
            'driverId': pit['driverId'],
            'stop': pit.get('stop', None),
            'lap': pit.get('lap', None),
            'time': pit.get('time', None),
            'duration': pit.get('duration', None)
            }
            pit_results.append(pit_info)
        return pit_results

def get_pit_results(start_year, end_year):
    all_results = []

    with ThreadPoolExecutor(max_workers=24) as executor:
        futures = []
        
        for year in range(start_year, end_year + 1):
            num_rounds = get_rounds_in_season(year)
            
            for round_num in range(1, num_rounds + 1):
                futures.append(executor.submit(fetch_pit_results, year, round_num))
        
        for future in futures:
            result = future.result()
            if result:
                all_results.extend(result)

    df_pit_results = pd.DataFrame(all_results)
    
    df_pit_results['duration'] = pd.to_numeric(df_pit_results['duration'], errors='coerce')
    
    return df_pit_results

In [159]:
def get_lap_times(start_year=2017, end_year=2024):
    all_laps = []
    for year in range(start_year, end_year + 1):
        num_rounds = get_rounds_in_season(year)
        for round_num in range(1, num_rounds + 1):
            lap_number = 1
            while True:
                url = f'http://ergast.com/api/f1/{year}/{round_num}/laps/{lap_number}.json'
                response = requests.get(url)
                data = response.json()['MRData']['RaceTable']['Races']
                if not data:
                    break
                laps = data[0].get('Laps', [])
                if not laps:
                    break
                for timing in laps[0]['Timings']:
                    lap_info = {
                        'season': year,
                        'round': round_num,
                        'lap': lap_number,
                        'position': int(timing['position']),
                        'driverId': timing['driverId'],
                        'time': timing['time']
                    }
                    all_laps.append(lap_info)
                
                lap_number += 1

    df_lap_times = pd.DataFrame(all_laps)
    return df_lap_times


In [ ]:
df_season_list = get_season_list(2017)
df_race_schedule = get_race_schedule(2017, 2024)
df_race_results = get_race_results(2017, 2024)
df_quali_results = get_qualifying_results(2017, 2024)
df_sprint_results = get_sprint_results(2021, 2024)
df_driver_standings = get_driver_standings(2017, 2024)
df_constructor_standings = get_constructor_standings(2017, 2024)
df_constructor_info = get_constructor_info()
df_driver_info = get_driver_info()
df_circuit_info = get_circuit_info()
df_finishing_status = get_finishing_status()
df_pit_results = get_pit_results(2017, 2024)
df_lap_times = get_lap_times(2017, 2024)

In [ ]:
df_season_list.to_csv('season_list.csv', index=False)
df_race_schedule.to_csv('race_schedule.csv', index=False)
df_race_results.to_csv('race_results.csv', index=False)
df_quali_results.to_csv('quali_results.csv', index=False)
df_sprint_results.to_csv('sprint_results.csv', index=False)
df_driver_standings.to_csv('driver_standings.csv', index=False)
df_constructor_standings.to_csv('constructor_standings.csv', index=False)
df_constructor_info.to_csv('constructor_info.csv', index=False)
df_driver_info.to_csv('driver_info.csv', index=False)
df_circuit_info.to_csv('circuit_info.csv', index=False)
df_finishing_status.to_csv('finishing_status.csv', index=False)
df_pit_results.to_csv('pit_results.csv', index=False)
df_lap_times.to_csv('lap_times.csv', index=False)